# Submit Nanoribbon SPM Calculation

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData

import ipywidgets as widgets
from IPython.display import display
import nglview

from collections import OrderedDict
# from aiida.workflows.user.scan_nanoribbons  import ScananoribbonWorkflow
# import os.path
# import sys

# import configparser
# from ase.io import read

## Step 1: Select structure from AiiDA database

In [ ]:
# query AiiDA database for structures
qb = QueryBuilder()
qb.append(StructureData)
   
#print qb.count()

all_nodes = [ node_tuple[0] for node_tuple in qb.iterall() ]
all_nodes.sort(key=lambda n: n.pk)

all_structs = OrderedDict()
all_structs["Select a Structure"] = None

for node in all_nodes:
    label = "PK: %d %s"%(node.pk, node.description)
    all_structs[label] = node

def on_struct_change(c):
    global atoms
    node = drop_struct.value
    atoms = node.get_ase()
    for i in viewer._ngl_component_ids:
        viewer.remove_component(i)
    viewer.add_component(nglview.ASEStructure(atoms))
    viewer.add_ball_and_stick()
    viewer.add_unitcell()
    viewer.center_view()

drop_struct = widgets.Dropdown(options=all_structs, value=None)
drop_struct.observe(on_struct_change, names='value')
viewer = nglview.NGLWidget()
display(widgets.VBox([drop_struct, viewer]))

## Step 2: Choose workflow parameters

In [ ]:
%%html

<!-- make widget labels wider -->
<style>
    .widget-label { min-width: 20ex !important; }
</style>

In [ ]:
l = widgets.Layout(width="400px")
maxkpt_bands = widgets.BoundedIntText(value=120, min=12, max=120, description='maxkpt_bands', layout=l)
maxkpt_cell  = widgets.BoundedIntText(value=30, min=3, max=30, description='maxkpt_cell', layout=l)
maxkpt_geo   = widgets.BoundedIntText(value=30, min=3, max=30, description='maxkpt_geo', layout=l)
maxkpt_scf   = widgets.BoundedIntText(value=60, min=6, max=60, description='maxkpt_scf', layout=l)
magauto      = widgets.Checkbox(description='Automatic mag.? ', layout=l)
refa0        = widgets.BoundedFloatText(value=2.461, min=2.461, max=2.481,step=0.01, description='refa0', layout=l)
range        = widgets.BoundedFloatText(value=2.0, min=2.0, max=4.0, step=0.1, description='range_sts_eV', layout=l)
delta_sts    = widgets.BoundedFloatText(value=0.05, min=0.02, max=0.1,step=0.01, description='delta_sts_eV', layout=l)
degauss_sts  = widgets.BoundedFloatText(value=0.075, min=0.03, max=0.15,step=0.01, description='degauss_sts_eV', layout=l)
torestart    = widgets.Checkbox(description='Restart calculation? ', layout=l)
fromw        = widgets.Dropdown(options=['SCF', 'RELAX', 'HARTREE', 'BANDS'], description='Restart from:', layout=l)
relax_pk     = widgets.BoundedIntText(value=0, min=0, max=30000, description='relax_pk', layout=l)
scf_pk       = widgets.BoundedIntText(value=0, min=0, max=30000, description='scf_pk', layout=l)
hartree_pk   = widgets.BoundedIntText(value=0, min=0, max=30000, description='hartree_pk', layout=l)
bands_pk     = widgets.BoundedIntText(value=0, min=0, max=30000, description='bands_pk', layout=l)
pw_codename  = widgets.Dropdown(options=['PW@HYPATIA','PW@DAINT'], description='PW CODE:', layout=l)
    
hartree_codename = widgets.Dropdown(options=['PP@HYPATIA','PP@DAINT'], description='HARTREE CODE:', layout=l)
sts_codename     = widgets.Dropdown(options=['PP_mod@HYPATIA','PP@DAINT'], description='PP CODE for sts:', layout=l)
pp_codename      = widgets.Dropdown(options=['PP@HYPATIA','PP@DAINT'], description='PP CODE:', layout=l)
bands_codename   = widgets.Dropdown(options=['BANDS@HYPATIA','BANDS@DAINT'], description='BANDS CODE:', layout=l)

num_mpi_procs_per_machine = widgets.BoundedIntText(value=16, min=1, max=44, description='#cores per node', layout=l)
nodes                     = widgets.BoundedIntText(value=1, min=1, max=20, description='#nodes', layout=l)
max_wallclock_seconds     = widgets.BoundedIntText(value=24*60*60, min=600, max=48*60*60, description='max seconds', layout=l)
pseudo_family             = widgets.Dropdown(options=['mypseudos','GTH','PAW'], description='pseudo family:', layout=l)

box = widgets.VBox([maxkpt_bands, maxkpt_cell,maxkpt_geo,maxkpt_scf,magauto,refa0,range,delta_sts,degauss_sts,
                    torestart,fromw,relax_pk,scf_pk,hartree_pk,bands_pk,pw_codename,hartree_codename,
                    sts_codename,pp_codename,bands_codename,
                    pseudo_family,
                    num_mpi_procs_per_machine,nodes,max_wallclock_seconds])

display(box)

## Step 3: Submit calculation

In [ ]:
def on_click_submit(b):
    print("TODO: implement job submission.")

btn_submit = widgets.Button(description="Submit Calculation")
btn_submit.on_click(on_click_submit)
display(btn_submit)

In [ ]:
def submit1():
    coordfname='fileout_from_sketch2xyz.xyz'

    thegeo=read(coordfname)
    strucformula=thegeo.get_chemical_formula()
    struclabel=strucformula+'  '+coordfname

    kpoints={}
    kpoints  ['maxkpt_bands']     = 12
    kpoints  ['maxkpt_cell' ]     = 3
    kpoints  ['maxkpt_geo'  ]     = 3
    kpoints  ['maxkpt_scf'  ]     = 6
    kpoints  ['refa0'       ]     = 2.4608613389

    sts_params = {}
    sts_params ['range']          = 2.0
    sts_params ['delta_sts']      = 0.05
    sts_params ['degauss_sts']    = 0.075

    res_params = {}
    res_params ['torestart']      = False
    res_params ['from']           = "SCF"
    res_params ['relax_pk']       = 0
    res_params ['scf_pk']         = 0
    res_params ['hartree_pk']     = 0
    res_params ['bands_pk']       = 0

    params = {'pw_codename'              : "PW@hypatia",
              'hartree_codename'         : "PP@hypatia",
              'sts_codename'             : "PP_mod@hypatia",
              'pp_codename'              : "PP@hypatia",
              'bands_codename'           : "bands5.2@hypatia",
              'num_mpiprocs_per_machine' : 16,
              'nodes'                    : 1,
              'max_wallclock_seconds'    : 24.0*60*60,
              'pseudo_family'            : "mypseudos",
              'struclabel'               : struclabel,
              'strucformula'             : strucformula,
              'inpcoord'                 : coordfname, 
              'mag_auto'                 : False,
              'kpoints'                  : kpoints,
              'sts'                      : sts_params,
              'restart'                  : res_params,
              'queue'                    : "short",
              }




In [ ]:
def submit2():
    if not params['restart']['torestart']:
     print "run from scratch"
     wf=ScananoribbonWorkflow(params=params)
     wf.store()
     wf.cell_opt()
     #wf.scf()
     #wf.bands()
     #wf.sts()
    else:
     if params['restart']['fromw']=='RELAX':
       wf=ScananoribbonWorkflow(params=params)
       wf.store()
       wf.relax()
     elif params['restart']['fromw']=='SCF':
       print "restart from SCF using relax" , params['restart']['relax_pk']
       wf=ScananoribbonWorkflow(params=params)
       wf.store()
       wf.scf()
     elif params['restart']['fromw']=='HARTREE':
       print "restart from HARTREE using " , params['restart']['scf_pk']
       wf=ScananoribbonWorkflow(params=params)
       wf.store()
       wf.plot_hartree()
     elif params['restart']['fromw']=='BANDS':
       print "restart from BANDS using " , params['restart']['scf_pk']
       wf=ScananoribbonWorkflow(params=params)
       wf.store()
       wf.bands()
     elif params['restart']['from']=='KS':
       print "restart from ks using " , params['restart']['bands_pk'], params['restart']['scf_pk']
       wf=ScananoribbonWorkflow(params=params)
       wf.store()
       wf.KS()
     elif params['restart']['fromw']=='STSF':
       wf=ScananoribbonWorkflow(params=params)
       wf.store()
       wf.stsf()
     elif params['restart']['fromw']=='STSE':
       wf=ScananoribbonWorkflow(params=params)
       wf.store()
       wf.stse()
     elif params['restart']['fromw']=='TOTMAG':
       wf=ScananoribbonWorkflow(params=params)
       wf.store()
       wf.totmag()
     else:
       print "wrong option"
       sys.exit()